In [ ]:
from torchvision import transforms
import torch
import numpy as np
import cv2
from matplotlib import pyplot as plt
from PIL import Image
import timeit
import pandas as pd
import gc

import utils.image as im
import utils.cam as cam
import utils.grabcut as gcut
import utils.voc as voc
import utils.json as json
from utils.VOCSegmentation import VOCSegmentation

In [ ]:
root_path = r'C:\Users\Nicol\Documents\EPFL\BA7\Project\Code\voc\\'

json_path = r'C:\Users\Nicol\Documents\EPFL\BA7\Project\Code\voc\json\\'

output_path = r'C:\Users\Nicol\Documents\EPFL\BA7\Project\Code\voc\sessions\4 Class grabcut-cam\\'

In [ ]:
camnet = cam.Cam()

In [ ]:
data_tbl = VOCSegmentation(root = root_path,
                           year = '2012',
                           image_set = 'trainval',
                           download = False,
                           transform = transforms.ToTensor(),
                           target_transform = transforms.ToTensor(),
                           transforms = None,
                           target = 'Class')

In [ ]:
data = iter(torch.utils.data.DataLoader(data_tbl,
                                        batch_size = 1,
                                        shuffle = False,
                                        num_workers = 0))

In [ ]:
annotations = json.open_json(json_path + "voc-class-annotations")
N = len(annotations)
annotations = iter(annotations.items())

classes = json.open_json(json_path + "voc-classes")

In [ ]:
IoU = np.array([])
time = np.array([])

for i in range(N):
    start = timeit.default_timer()
    gc.collect()
    
    img, sgm = next(data)
    img, sgm = torch.squeeze(img), torch.squeeze(sgm)

    img_pil = img
    img_cv2 = im.pil_to_cv2(img.numpy())
    sgm     = im.f1_to_f255(sgm.numpy())
    
    name, annots = next(annotations)
    
    for c in annots:
        
        img_cam = camnet.get_top(img_pil, c)
        img_cam = cam.cam_process(img_pil, img_cam)
        gcmask_cam = cam.cam_to_gcmask(img_cam, a = 0.0, b = 0.2, c = 1.0)
        _, _, pred = gcut.grabcut(img_cv2, gcmask_cam, mode = 'MASK')
        
        true = voc.true_mask(sgm, classes.index(c) + 1, dilation = 0)
        
        IoU = np.append(IoU, voc.IoU_acc_undef(true, pred, sgm))
        
    stop = timeit.default_timer()
    time = np.append(time, stop - start)
    if i % 20 == 0:
        print(f'Image nb {i}')
        print(f'Time spent               = ' + voc.time_str(np.sum(time)))
        print(f'Estimated time remaining = ' + voc.time_str(np.mean(time) * (N - 1 - i)))
        print(f'Mean IoU                 =  {np.mean(IoU) :.3f}')
        print()

In [ ]:
print(f'Mean IoU = {np.mean(IoU):.3f}')

In [ ]:
df = pd.read_csv(r'C:\Users\Nicol\Documents\EPFL\BA7\Project\Code\voc\json\voc-class-annotations.csv')

In [ ]:
df['IoU'] = IoU

In [ ]:
df.to_csv(output_path + 'IoU.csv', index = False)